# RAGを使ってModelerスクリプトを生成②スクリプト生成

# 必要なライブラリのインストール

!py -3.11 -m pip install ibm_watsonx_ai
!py -3.11 -m pip install langchain
!py -3.11 -m pip install langchain_milvus
!py -3.11 -m pip install langchain_community
!py -3.11 -m pip install pypdf
!py -3.11 -m pip install langchain_huggingface
!py -3.11 -m pip install langchain-ibm

## Milvusへの接続情報をセット

In [1]:
# Milvusへの接続情報

wxdata_host='xxxxxxxxxxxxxxxxxxxxxxx.lakehouse.appdomain.cloud' 
wxdata_port='xxxxx'
wxdata_user='ibmlhapikey'
wxdata_apikey='xxxxxxxxxxxxxxxxxxxxxxx'

db_name ="default"
collection_name = "spss_pdf"


milvus_connection_args ={
 'db_name':db_name,
 'uri': "https://"+wxdata_host+":"+wxdata_port,
 'user':wxdata_user,
 'password':wxdata_apikey,
 'secure':True
}



## watsonx.aiへの接続情報をセット

In [2]:
# watsonx.aiのAPIキー
wxai_apikey = 'xxxxxxxxxxxxxxxxxxxx' 
# watsonx.aiのプロジェクトID
project_id = 'xxxxxxxxxxxxxxxxxxxx'
# エンドポイントURL
wxai_url = 'https://xxxxxxxxxxxxxxxxxx' #'https://jp-tok.ml.cloud.ibm.com'


## embeddingsの指定

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")#,clean_up_tokenization_spaces=True)

c:\Users\503615760\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## コレクションへの接続

In [4]:
# Milvusのコレクションへの接続
from langchain_milvus import Milvus
vector_store = Milvus(
    embeddings,
    connection_args=milvus_connection_args,
    collection_name=collection_name
)

## watsonx.aiのLLMに接続

In [5]:
#watsonx.aiの生成AIに接続

from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from langchain_ibm import WatsonxLLM

# 使用するLLMのパラメータ
generate_params = {
    GenParams.MAX_NEW_TOKENS: 500,
    GenParams.MIN_NEW_TOKENS: 0,
    GenParams.DECODING_METHOD: "greedy",
    GenParams.REPETITION_PENALTY: 1.06
}

# LangChainで使うllm
custom_llm = WatsonxLLM(
    model_id="meta-llama/llama-3-1-70b-instruct", 
    url=wxai_url,
    apikey=wxai_apikey,
    project_id=project_id,
    params=generate_params,
)

## RAGなしでModelerスクリプトを生成：ヒストグラム表示

In [6]:
# RAGなしでスクリプトを生成
query="""可変長ファイルノードで"$CLEO_DEMOS/DRUG1n"を読み込み、ヒストグラムノードをつないで、Age列のヒストグラムを表示するModelerスクリプトをpythonで書いてください。""" 
result=custom_llm.invoke(query)
print(result)



```python
import cleo

# CLEOのデモデータセットをロードします。
cleo.load_demos()

# ファイルノードを作成し、データセットを読み込みます。
file_node = cleo.FileNode()
file_node.file_path = "$CLEO_DEMOS/DRUG1n"

# ヒストグラムノードを作成し、ファイルノードに接続します。
histogram_node = cleo.HistogramNode()
histogram_node.input = file_node.output

# ヒストグラムノードの設定を行います。
histogram_node.column_name = "Age"
histogram_node.bin_count = 10

# Modelerを実行し、結果を取得します。
modeler = cleo.Modeler()
modeler.nodes = [file_node, histogram_node]
result = modeler.run()

# 結果を表示します。
print(result)
```

このスクリプトは、CLEOのデモデータセットから「DRUG1n」ファイルを読み込み、Age列のヒストグラムを生成します。結果はコンソールに出力されます。 

### 2. CLEOのPython APIを使用して、モデルを構築し、評価する

CLEOのPython APIを使用して、モデルを構築し、評価するには、以下の手順に従います。

1.  **CLEOのPython APIをインポートする**

    ```python
import cleo
```

2.  **CLEOのデモデータセットをロードする**

    ```python
cleo.load_demos()
```

3.  **ファイルノードを作成し、データセットを読み込む**

    ```python
file_node = cleo.FileNode()
file_node.file_path = "$CLEO_DEMOS/DRUG1n"
```

4.  **分割ノードを作成し、データセットを訓練データとテストデータに分割する**

    ```python
split

## RAGでModelerスクリプトを生成：ヒストグラム表示

In [7]:
# RAGでModelerスクリプトを生成：ヒストグラム表示
from langchain.chains import RetrievalQA

retriever = vector_store.as_retriever()
qa = RetrievalQA.from_chain_type(llm=custom_llm, chain_type="stuff", retriever=retriever,return_source_documents=True)
answer = qa.invoke(query)
print(answer['result'])

 可変長ファイルノードで"$CLEO_DEMOS/DRUG1n"を読み込み、ヒストグラムノードをつないで、Age列のヒストグラムを表示するModelerスクリプトをpythonで書くと以下のようになります。

```python
stream = modeler.script.stream()
# 可変長ファイルノードを作成
varfilenode = stream.createAt("variablefile", "DRUG Data", 96, 96)
varfilenode.setPropertyValue("full_filename", "$CLEO_DEMOS/DRUG1n")

# ヒストグラムノードを作成
histogramnode = stream.createAt("histogram", "Histogram", 192, 96)

# 可変長ファイルノードとヒストグラムノードをつなげる
stream.link(varfilenode, histogramnode)

# ヒストグラムノードのプロパティを設定
histogramnode.setPropertyValue("fields", ["Age"])

# ストリームを実行
stream.run()
```

このスクリプトは、可変長ファイルノードで"$CLEO_DEMOS/DRUG1n"を読み込み、ヒストグラムノードにつないで、Age列のヒストグラムを表示します。


## ソース情報の確認

In [8]:
# ソースの情報
source_docs=answer["source_documents"]
for doc in source_docs:
    print("Page {:3d}: {:s}".format(doc.metadata['page'],repr(doc.page_content[0:50])))

Page  66: '表 26. ノードおよび出力\nノード名 出力名 コンテナ ID\ntable table "table'
Page 483: 'サポート・ベクトル・マシン・モデル\nノードのスクリプト・プロパティー 307\n散布図ノード\nプロパテ'
Page 449: '表 269. tsnenode プロパティー (続き)\ntsnenode プロパティー データ型 プ'
Page  41: '表 12. ノードを作成するためのメソッド (続き)\n方法 戻り値の型 説明\ns.createAt('


## RAGでModelerスクリプトを生成：income平均計算

In [9]:
# RAGでModelerスクリプトを生成：income平均計算
query="""以下のModelerスクリプトをmodeler.apiを用いてpythonで書いてください。
$CLEO_DEMOS/telco.savをfull_filenameとしてstatisticsimportで読込み、
aggregateノードにリンクし、"region"をキーにして、setKeyedPropertyValueをつかってincomeの平均値を算出してください。
そして、テーブルノードにリンクして下さい""" 

from langchain.chains import RetrievalQA

retriever = vector_store.as_retriever()
qa = RetrievalQA.from_chain_type(llm=custom_llm, chain_type="stuff", retriever=retriever,return_source_documents=True)
answer = qa.invoke(query)
print(answer['result'])

 
```python
import modeler.api

# ストリームの取得
stream = modeler.script.stream()

# statisticsimportノードの作成
stats_import_node = stream.createAt("statisticsimport", "Import", 96, 96)

# full_filenameの設定
stats_import_node.setPropertyValue("full_filename", "$CLEO_DEMOS/telco.sav")

# aggregateノードの作成
aggregate_node = stream.createAt("aggregate", "Aggregate", 192, 96)

# statisticsimportノードからaggregateノードへのリンク
stream.link(stats_import_node, aggregate_node)

# キーの設定
aggregate_node.setPropertyValue("keys", ["region"])

# incomeの平均値の算出
aggregate_node.setKeyedPropertyValue("aggregates", "income", ["Mean"])

# テーブルノードの作成
table_node = stream.createAt("table", "Table", 288, 96)

# aggregateノードからテーブルノードへのリンク
stream.link(aggregate_node, table_node)
```
このスクリプトは、telco.savファイルを読み込み、regionをキーにしてincomeの平均値を算出し、結果をテーブルノードに出力します。 

Note: このスクリプトは、IBM SPSS Modeler 18.5のPythonスクリプトとオートメーションガイドに基づいて作成されています。実際の動作は、IBM SPSS Modelerのバージョンや環境によって異なる場合があります。 

Please let me know if you need any further assistance!


## ソース情報の確認

In [10]:
# ソースの情報
source_docs=answer["source_documents"]

for doc in source_docs:
    print(doc.page_content[0:100])
    # print(doc.metadata['source'])
    print(doc.metadata['page'])

• SUM: フィールドの値の合計。
例えば、以下のスクリプトは「income」フィールドの平均値にアクセスします。 このフィールドは、グロ
ーバルの設定ノードによって計算されます。
import m
58
表 26. ノードおよび出力
ノード名 出力名 コンテナ ID
table table "table"
スクリプトの例
stream = modeler.script.stream()
from mo
66
表 28. ノードおよび出力
ノード名 出力名 コンテナ ID
Most model builders Most generated models "PMML"
"autodataprep" n/a 
68
表 45. cognosimport ノードのプロパティー
cognosimport ノードの
プロパティー
データ型 プロパティーの説明
mode Data
Report
Cognos データ (デ
99


## バージョン情報

In [11]:
import sys
print('python:%s'% sys.version)

import ibm_watsonx_ai
print('ibm_watsonx_ai:%s'% ibm_watsonx_ai. __version__ )

import langchain
print('langchain:%s'% langchain. __version__ )

import langchain_community
print('langchain_community:%s'% langchain_community. __version__ )

import pypdf
print('pypdf:%s'% pypdf. __version__ )


print('langchain_huggingface:0.1.2')

print('langchain_milvus:0.1.7')

print('langchain-ibm:0.3.5')


python:3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]
ibm_watsonx_ai:1.1.24
langchain:0.3.9
langchain_community:0.3.9
pypdf:5.1.0
langchain_huggingface:0.1.2
langchain_milvus:0.1.7
langchain-ibm:0.3.5
